In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/brca_cancer (1).csv')

In [ ]:
df.head()

,Unnamed: 0,gene_id,gene_name,gene_type,unstranded,stranded_first,stranded_second,tpm_unstranded,fpkm_unstranded,fpkm_uq_unstranded,class
0,4,ENSG00000000003.15,TSPAN6,protein_coding,5175,2582,2594,58.4397,16.4625,16.0650,0
1,5,ENSG00000000005.6,TNMD,protein_coding,85,52,33,2.9499,0.8310,0.8109,0
2,6,ENSG00000000419.13,DPM1,protein_coding,2406,1194,1212,102.1078,28.7639,28.0692,0
3,7,ENSG00000000457.14,SCYL3,protein_coding,2222,1629,1631,16.5362,4.6583,4.5458,0
4,8,ENSG00000000460.17,C1orf112,protein_coding,476,808,782,4.0842,1.1505,1.1227,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121320 entries, 0 to 121319
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          121320 non-null  int64  
 1   gene_id             121320 non-null  object 
 2   gene_name           121320 non-null  object 
 3   gene_type           121320 non-null  object 
 4   unstranded          121320 non-null  int64  
 5   stranded_first      121320 non-null  int64  
 6   stranded_second     121320 non-null  int64  
 7   tpm_unstranded      121320 non-null  float64
 8   fpkm_unstranded     121320 non-null  float64
 9   fpkm_uq_unstranded  121320 non-null  float64
 10  class               121320 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 10.2+ MB


In [ ]:
dfe=df.loc[:,['gene_name','tpm_unstranded','class']]

In [ ]:
dfe.head()

,gene_name,tpm_unstranded,class
0,TSPAN6,58.4397,0
1,TNMD,2.9499,0
2,DPM1,102.1078,0
3,SCYL3,16.5362,0
4,C1orf112,4.0842,0


In [ ]:
len(dfe['gene_name'].unique()) # Total number of unique genes in an organ

59427

In [ ]:
dfe['gene_name'].value_counts()

gene_name
Y_RNA          1512
Metazoa_SRP     340
U3              100
U6               66
SNORA70          54
               ... 
CYS1              2
PLPP6             2
KLRC2             2
KLRC3             2
AP006621.6        2
Name: count, Length: 59427, dtype: int64

In [ ]:
dfe[dfe['gene_name']=='PIK3CA']  # PIK3CA Gene

,gene_name,tpm_unstranded,class
5204,PIK3CA,10.6744,0
65864,PIK3CA,6.7967,1


In [ ]:
dfe[dfe['gene_name']=='TP53']

,gene_name,tpm_unstranded,class
8124,TP53,55.8894,0
68784,TP53,42.4685,1


In [ ]:
dfe[dfe['gene_name']=='BRCA1']

,gene_name,tpm_unstranded,class
310,BRCA1,4.8549,0
60970,BRCA1,3.0269,1


In [ ]:
dfe[dfe['gene_name']=='NF1']

,gene_name,tpm_unstranded,class
17196,NF1,19.8343,0
77856,NF1,7.0352,1


In [ ]:
#Label Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [ ]:
# Encode labels in column 'species'
dfe['gene_name']=encoder.fit_transform(dfe['gene_name'])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y=dfe['class']
X=dfe.drop(['class'],axis=1)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [ ]:
dtree=DecisionTreeClassifier()

In [ ]:
dtree.fit(X_train,y_train)

DecisionTreeClassifier()

In [ ]:
y_predictions=dtree.predict(X_test)

In [ ]:
print(classification_report(y_test,y_predictions))

              precision    recall  f1-score   support

           0       0.38      0.40      0.39     18210
           1       0.35      0.33      0.34     18186

    accuracy                           0.37     36396
   macro avg       0.37      0.37      0.37     36396
weighted avg       0.37      0.37      0.37     36396



In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import random

In [ ]:
gene_counts = df.groupby('gene_name')['class'].nunique()
genes_with_both_classes = gene_counts[gene_counts == 2].index
filtered_df = df[df['gene_name'].isin(genes_with_both_classes)]

encoder = LabelEncoder()
filtered_df['gene_name_encoded'] = encoder.fit_transform(filtered_df['gene_name'])

gene_pool = filtered_df['gene_name_encoded'].unique()

y = filtered_df['class']
X = filtered_df[['gene_name_encoded', 'tpm_unstranded']]

In [ ]:
def fitness_function(selected_gene_indices):
    selected_genes = [gene_pool[i] for i in selected_gene_indices]
    X_selected = X[X['gene_name_encoded'].isin(selected_genes)]

    X_selected = X_selected.drop_duplicates()
    y_selected = y[X_selected.index]

    X_train, X_test, y_train, y_test = train_test_split(X_selected, y_selected, test_size=0.3, random_state=101)

    dtree = DecisionTreeClassifier()
    dtree.fit(X_train, y_train)
    y_pred = dtree.predict(X_test)

    return accuracy_score(y_test, y_pred)

def initialize_population(size, gene_pool_size, chromosome_length=30):
    return [np.random.choice(gene_pool_size, size=chromosome_length, replace=False) for _ in range(size)]

def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1)-1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def mutate(chromosome, mutation_rate, gene_pool_size):
    if random.random() < mutation_rate:
        mutation_point = np.random.randint(0, len(chromosome))
        new_gene = np.random.randint(0, gene_pool_size)
        chromosome[mutation_point] = new_gene
    return chromosome

def genetic_algorithm(gene_pool, population_size=100, generations=50, mutation_rate=0.01):
    gene_pool_size = len(gene_pool)
    population = initialize_population(population_size, gene_pool_size)

    for _ in range(generations):
        fitness_scores = np.array([fitness_function(individual) for individual in population])

        sorted_indices = np.argsort(fitness_scores)[::-1]
        population = [population[index] for index in sorted_indices[:population_size//2]]

        next_generation = []
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate, gene_pool_size)
            child2 = mutate(child2, mutation_rate, gene_pool_size)
            next_generation.extend([child1, child2])

        population = next_generation

    best_individual_index = np.argmax([fitness_function(individual) for individual in population])
    best_genes_indices = population[best_individual_index]
    best_genes = [gene_pool[i] for i in best_genes_indices]

    return encoder.inverse_transform(best_genes)

In [ ]:
selected_genes = genetic_algorithm(gene_pool)
print("Selected genes:")
print(selected_genes)

Selected genes:
['PITX3' 'AC034229.1' 'HMGB3P7' 'UBN1' 'AL353586.1' 'MMP25-AS1' 'MEIS2'
 'TGS1' 'RAB30-DT' 'LINC02108' 'FLNB-AS1' 'TMEM178B' 'AC231759.1'
 'AL845552.1' 'SRSF10' 'RN7SKP30' 'COL9A3' 'AC107308.1' 'AC073188.2'
 'CYP3A54P' 'RN7SL727P' 'AC108516.2' 'SUB1' 'CYP2W1' 'PRRC2B' 'FAM221A'
 'TRBV7-5' 'TRAJ26' 'FAM83A-AS1' 'AC133485.6']


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

filtered_df = df[df['gene_name'].isin(selected_genes)]

filtered_df = filtered_df.groupby('gene_name').filter(lambda x: x['class'].nunique() == 2)

if len(filtered_df) == 0:
    raise ValueError("No genes with representation in both classes were found. Check your selected genes list and dataset.")

X = filtered_df[['tpm_unstranded']]
y = filtered_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

y_pred = dtree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91        10
           1       1.00      0.75      0.86         8

    accuracy                           0.89        18
   macro avg       0.92      0.88      0.88        18
weighted avg       0.91      0.89      0.89        18

